In [ ]:
!pip install streamlit

     |████████████████████████████████| 7.8MB 11.2MB/s 
     |████████████████████████████████| 174kB 49.6MB/s 
     |████████████████████████████████| 112kB 55.8MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 4.2MB 43.2MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 122kB 56.9MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13476 sha256=9f14ac4c76c98b09622271eed94982eb20357d4b6bc28765c20e9d05c7caca6a
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [ ]:
#https://stackoverflow.com/a/57480696/6868740

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
%%writefile app.py

import cv2
import streamlit as st
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import base64


#https://docs.streamlit.io/en/stable/api.html?highlight=beta_page_config#streamlit.set_page_config
st.set_page_config(
page_title="Text detection App",
page_icon="🧊",
layout="wide")

st.set_option('deprecation.showfileUploaderEncoding', False)
@st.cache(allow_output_mutation=True)
def load_model(model_type):
  if model_type == 'keras':
    #https://github.com/keras-team/keras/issues/5916
    model= tf.keras.models.load_model('/content/gdrive/MyDrive/saved_model.h5', custom_objects={'dice_coef_loss': dice_coef_loss,'rbox_loss':rbox_loss})
  else:
    model = tf.lite.Interpreter(model_path="/content/gdrive/MyDrive/tflite_quant_model_Float.tflite")
  return model


st.write('''
        # Natural Scene text detection
        ''')


option = st.selectbox('Which model you want to use?',
                      ('keras', 'quantized'))

st.write('Model selected now :', option)


file= st.file_uploader('please upload an image with text', type=['jpg', 'png'])

def resize_image_1(im, max_side_len=2400):
    '''
    resize image to a size multiple of 32 which is required by the network
    :param im: the resized image
    :param max_side_len: limit of max image size to avoid out of memory in gpu
    :return: the resized image and the resize ratio
    '''
    h, w, _ = im.shape
    #print(h,w)
    resize_w = w
    resize_h = h

    # limit the max side
    if max(resize_h, resize_w) > max_side_len:
        ratio = float(max_side_len) / resize_h if resize_h > resize_w else float(max_side_len) / resize_w
    else:
        ratio = 1.
    resize_h = int(resize_h * ratio)
    resize_w = int(resize_w * ratio)

    resize_h = resize_h if resize_h % 32 == 0 else (resize_h // 32) * 32
    resize_w = resize_w if resize_w % 32 == 0 else (resize_w // 32) * 32
    im = cv2.resize(im, (int(resize_w), int(resize_h)))

    ratio_h = resize_h / float(h)
    ratio_w = resize_w / float(w)

    return im, (ratio_h, ratio_w)


def detect(score_map, geo_map, score_map_thresh=0.8, box_thresh=0.1, nms_thres=0.2):
    '''
    restore text boxes from score map and geo map
    :param score_map:
    :param geo_map:
    :param score_map_thresh: threshhold for score map
    :param box_thresh: threshhold for boxes
    :param nms_thres: threshold for nms
    :return:
    '''
    
    if len(score_map.shape) == 4:
        score_map = score_map[0, :, :, 0]
        geo_map = geo_map[0, :, :, ]
    # filter the score map
    xy_text = np.argwhere(score_map > score_map_thresh)
    # sort the text boxes via the y axis
    xy_text = xy_text[np.argsort(xy_text[:, 0])]
    # restore
    #start = time.time()
    text_box_restored = restore_rectangle_rbox(xy_text[:, ::-1]*4, geo_map[xy_text[:, 0], xy_text[:, 1], :]) # N*4*2
    boxes = np.zeros((text_box_restored.shape[0], 9), dtype=np.float32)
    boxes[:, :8] = text_box_restored.reshape((-1, 8))
    boxes[:, 8] = score_map[xy_text[:, 0], xy_text[:, 1]]
    #timer['restore'] = time.time() - start
    # nms part
    #start = time.time()
    # boxes = nms_locality.nms_locality(boxes.astype(np.float64), nms_thres)
    boxes = nms_locality(boxes.astype('float32'), nms_thres)
    #timer['nms'] = time.time() - start

    if boxes.shape[0] == 0:
        return None

    # here we filter some low score boxes by the average score map, this is different from the orginal paper
    for i, box in enumerate(boxes):
        mask = np.zeros_like(score_map, dtype=np.uint8)
        cv2.fillPoly(mask, box[:8].reshape((-1, 4, 2)).astype(np.int32) // 4, 1)
        boxes[i, 8] = cv2.mean(score_map, mask)[0]
    boxes = boxes[boxes[:, 8] > box_thresh]

    return boxes


def sort_poly(p):
    min_axis = np.argmin(np.sum(p, axis=1))
    p = p[[min_axis, (min_axis+1)%4, (min_axis+2)%4, (min_axis+3)%4]]
    if abs(p[0, 0] - p[1, 0]) > abs(p[0, 1] - p[1, 1]):
        return p
    else:
        return p[[0, 3, 2, 1]]
        
def restore_rectangle_rbox(origin, geometry):
    d = geometry[:, :4]
    angle = geometry[:, 4]
    # for angle > 0
    origin_0 = origin[angle >= 0]
    d_0 = d[angle >= 0]
    angle_0 = angle[angle >= 0]
    if origin_0.shape[0] > 0:
        p = np.array([np.zeros(d_0.shape[0]), -d_0[:, 0] - d_0[:, 2],
                      d_0[:, 1] + d_0[:, 3], -d_0[:, 0] - d_0[:, 2],
                      d_0[:, 1] + d_0[:, 3], np.zeros(d_0.shape[0]),
                      np.zeros(d_0.shape[0]), np.zeros(d_0.shape[0]),
                      d_0[:, 3], -d_0[:, 2]])
        p = p.transpose((1, 0)).reshape((-1, 5, 2))  # N*5*2

        rotate_matrix_x = np.array([np.cos(angle_0), np.sin(angle_0)]).transpose((1, 0))
        rotate_matrix_x = np.repeat(rotate_matrix_x, 5, axis=1).reshape(-1, 2, 5).transpose((0, 2, 1))  # N*5*2

        rotate_matrix_y = np.array([-np.sin(angle_0), np.cos(angle_0)]).transpose((1, 0))
        rotate_matrix_y = np.repeat(rotate_matrix_y, 5, axis=1).reshape(-1, 2, 5).transpose((0, 2, 1))

        p_rotate_x = np.sum(rotate_matrix_x * p, axis=2)[:, :, np.newaxis]  # N*5*1
        p_rotate_y = np.sum(rotate_matrix_y * p, axis=2)[:, :, np.newaxis]  # N*5*1

        p_rotate = np.concatenate([p_rotate_x, p_rotate_y], axis=2)  # N*5*2
        p3_in_origin = origin_0 - p_rotate[:, 4, :]
        new_p0 = p_rotate[:, 0, :] + p3_in_origin  # N*2
        new_p1 = p_rotate[:, 1, :] + p3_in_origin
        new_p2 = p_rotate[:, 2, :] + p3_in_origin
        new_p3 = p_rotate[:, 3, :] + p3_in_origin

        new_p_0 = np.concatenate([new_p0[:, np.newaxis, :], new_p1[:, np.newaxis, :],
                                  new_p2[:, np.newaxis, :], new_p3[:, np.newaxis, :]], axis=1)  # N*4*2
    else:
        new_p_0 = np.zeros((0, 4, 2))
    # for angle < 0
    origin_1 = origin[angle < 0]
    d_1 = d[angle < 0]
    angle_1 = angle[angle < 0]
    if origin_1.shape[0] > 0:
        p = np.array([-d_1[:, 1] - d_1[:, 3], -d_1[:, 0] - d_1[:, 2],
                      np.zeros(d_1.shape[0]), -d_1[:, 0] - d_1[:, 2],
                      np.zeros(d_1.shape[0]), np.zeros(d_1.shape[0]),
                      -d_1[:, 1] - d_1[:, 3], np.zeros(d_1.shape[0]),
                      -d_1[:, 1], -d_1[:, 2]])
        p = p.transpose((1, 0)).reshape((-1, 5, 2))  # N*5*2

        rotate_matrix_x = np.array([np.cos(-angle_1), -np.sin(-angle_1)]).transpose((1, 0))
        rotate_matrix_x = np.repeat(rotate_matrix_x, 5, axis=1).reshape(-1, 2, 5).transpose((0, 2, 1))  # N*5*2

        rotate_matrix_y = np.array([np.sin(-angle_1), np.cos(-angle_1)]).transpose((1, 0))
        rotate_matrix_y = np.repeat(rotate_matrix_y, 5, axis=1).reshape(-1, 2, 5).transpose((0, 2, 1))

        p_rotate_x = np.sum(rotate_matrix_x * p, axis=2)[:, :, np.newaxis]  # N*5*1
        p_rotate_y = np.sum(rotate_matrix_y * p, axis=2)[:, :, np.newaxis]  # N*5*1

        p_rotate = np.concatenate([p_rotate_x, p_rotate_y], axis=2)  # N*5*2

        p3_in_origin = origin_1 - p_rotate[:, 4, :]
        new_p0 = p_rotate[:, 0, :] + p3_in_origin  # N*2
        new_p1 = p_rotate[:, 1, :] + p3_in_origin
        new_p2 = p_rotate[:, 2, :] + p3_in_origin
        new_p3 = p_rotate[:, 3, :] + p3_in_origin

        new_p_1 = np.concatenate([new_p0[:, np.newaxis, :], new_p1[:, np.newaxis, :],
                                  new_p2[:, np.newaxis, :], new_p3[:, np.newaxis, :]], axis=1)  # N*4*2
    else:
        new_p_1 = np.zeros((0, 4, 2))
    return np.concatenate([new_p_0, new_p_1])
from shapely.geometry import Polygon 
def nms_locality(polys, threshold=0.3):
    S=[]
    p=None
    
    for g in polys:
        if p is not None and iOu(g,p) > threshold:
            p= weighted_merge(g,p)
        else:
            if p is not None:
                S.append(p)
            p=g
    
    if p is not None:
        S.append(p)
    
    if len(S) == 0:
        return np.array([])
    
    return standard_NMS(np.array(S), threshold)


#defining intersection over union fn 
def iOu(g,p):
    g= Polygon(g[:8].reshape((4,2)))
    p= Polygon(p[:8].reshape((4,2)))
    
    if not g.is_valid  or not p.is_valid:
        return 0
    
    inter= Polygon(g).intersection(Polygon(p)).area
    union= g.area + p.area - inter
    
    if union==0:
        return 0
    else:
        return inter/union


#defining weighted_merge fn
def weighted_merge(g,p):
    g[:8]= (g[:8]*g[8] + p[:8]*p[8])/(g[8]+p[8])
    g[8]= g[8]+p[8]
    return g


def standard_NMS(S,thresh):
    indx= np.argsort(S[:,8])[::-1] # [:,8] because we want to sort by score. And after that change it in descending order using -1
    keep=[]
    while len(indx)>0:
        max_score_idx= indx[0]# 0 bcoz now it is in descending order, max is at 1st pos
        keep.append(max_score_idx)
        ovr= np.array([iOu(S[max_score_idx], S[i]) for i in indx[1:]])
        idx= np.where(ovr <= thresh)[0]
        indx = indx[idx+1]
    return S[keep]
        


def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1. - dice_coef(y_true, y_pred)
    


def rbox_loss(y_true, y_pred):
    d1_gt, d2_gt, d3_gt, d4_gt, theta_gt = tf.split(value=y_true, num_or_size_splits=5, axis=3)
    d1_pred, d2_pred, d3_pred, d4_pred, theta_pred = tf.split(value=y_pred, num_or_size_splits=5, axis=3)
    area_gt = (d1_gt + d3_gt) * (d2_gt + d4_gt)
    area_pred = (d1_pred + d3_pred) * (d2_pred + d4_pred)
    w_union = tf.minimum(d2_gt, d2_pred) + tf.minimum(d4_gt, d4_pred)
    h_union = tf.minimum(d1_gt, d1_pred) + tf.minimum(d3_gt, d3_pred)
    area_intersect = w_union * h_union
    area_union = area_gt + area_pred - area_intersect
    L_AABB = -tf.math.log((area_intersect + 1.0)/(area_union + 1.0))
    L_theta = 1 - tf.math.cos(theta_pred - theta_gt)
    L_g = L_AABB + 20 * L_theta
    ll= tf.reduce_mean(L_g)
    return ll





# image should be given as input after reading i.e. as image data or image array
def final_fun_1(image, model_type):
  img_resized, (ratio_h, ratio_w) = resize_image_1(image)
  img_resized = (img_resized / 127.5) - 1

  model= load_model(model_type)
  # feed image into model
  if model_type=='keras':
      score_map, geo_map = model.predict(img_resized[np.newaxis, :, :, :])

  else:
      #here model is interpreter
      img_resized = img_resized[np.newaxis, :, :, :].astype('float32')

      model.resize_tensor_input(0, np.array(img_resized.shape))

      model.allocate_tensors()

      # Get input and output tensors.
      input_details = model.get_input_details()
      output_details = model.get_output_details()
      # Test model on random input data.
      input_shape = input_details[0]['shape']
      model.set_tensor(input_details[0]['index'], img_resized)
      
      model.invoke()
      # The function `get_tensor()` returns a copy of the tensor data.
      # Use `tensor()` in order to get a pointer to the tensor.
      score_map = model.get_tensor(output_details[0]['index']) #1st output
      geo_map = model.get_tensor(output_details[1]['index']) #2nd output


  boxes = detect(score_map=score_map, geo_map=geo_map, nms_thres=0.08, box_thresh=0.1, score_map_thresh=0.6)

  if boxes is not None:
      boxes = boxes[:, :8].reshape((-1, 4, 2))
      boxes[:, :, 0] /= ratio_w
      boxes[:, :, 1] /= ratio_h

      for box in boxes:
        # to avoid submitting errors/ very small boxes  
        box = sort_poly(box.astype(np.int32))
        if np.linalg.norm(box[0] - box[1]) < 10 or np.linalg.norm(box[3]-box[0]) < 10:
            continue
        image=cv2.polylines(image, [box.astype(np.int32).reshape((-1, 1, 2))], True, color=(0, 255, 0), thickness=2)
  
  return image



#     return 2. * intersection / (K.sum((y_true),-1) + K.sum((y_pred),-1) + smooth)



def display_img(raw_image):
  plt.figure(figsize=(14,12))
  plt.imshow(raw_image) 
  plt.show()


#https://docs.streamlit.io/en/stable/api.html#placeholders-help-and-options
imageLocation = st.empty()

col1, col2 = imageLocation.beta_columns(2)

def import_and_detect(img, model_type):
    # Convert the file to an opencv image.
    #https://github.com/streamlit/streamlit/issues/888
    file_bytes = np.asarray(bytearray(img.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1)
    
    col1.header("Original")
    col1.image(image[:,:,::-1], width=400)

    # Now do something with the image! For example, let's display it:
    col2.header("Text detected")
    #col2.image(image, use_column_width=True)
    col2.image(
            "https://media.tenor.com/images/80cb16bb74ed9027ea1b25d077ce6d97/tenor.gif", # I prefer to load the GIFs using GIPHY
            width=400)
    #imageLocation.image('', channels='BGR', width= 600)
    imga=image[:, :, ::-1].copy()
    eval_image= final_fun_1(imga,model_type)
    return image,eval_image

if file is None:
    st.text('Please upload an image')
else:
    img,eval_image= import_and_detect(file,option)
    #col2.empty()
    #replace placeholer with new beta colms
    col1, col2 = imageLocation.beta_columns(2)
    col1.header("Original")
    col1.image(img[:,:,::-1], width=400)
    col2.header("text detected")
    col2.image(eval_image, use_column_width=True, channels="RGB", width= 400)
    
    #imageLocation.image([img[:,:,::-1],eval_image], ,width= 400)
    st.success('Here is text detected')
    





Writing app.py


In [ ]:
!pip install pyngrok

     |████████████████████████████████| 747kB 13.1MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19262 sha256=24836626c281ca907eba18d634eed6b1075c2d0a85a0141b5e0b54651109a0c1
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [ ]:
#https://medium.com/@jcharistech/how-to-run-streamlit-apps-from-colab-29b969a1bdfc
#https://www.youtube.com/watch?v=Q1NC3NbmVlc
from pyngrok import ngrok

!ngrok authtoken 1u9aQb3HKxh0k627Nj3AIoM5lGw_6t1xbxgs7yzrCaX5urUtq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#https://discuss.streamlit.io/t/streamlit-with-colab-and-pyngrok-failed-to-complete-tunnel-connection-version-issue/7299

url= ngrok.connect(port=80)
print(url)
!nohup streamlit run --server.port 80 app.py

NgrokTunnel: "http://e88b08aeac93.ngrok.io" -> "http://localhost:80"
nohup: ignoring input and appending output to 'nohup.out'


In [ ]:
!cat /content/nohup.out


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.204.75.64:8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8502
  External URL: http://35.204.75.64:8502


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8503
  External URL: http://35.204.75.64:8503

  Stopping...
  Stopping...
  Stopping...

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.204.75.64:8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8502
  External URL: http://35.204.75.64:8502


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8503
  External URL: http://35.204.75.64:8503



In [ ]:
!top

=top - 05:39:47 up  1:17,  0 users,  load average: 0.03, 0.05, 0.00
Tasks:  23 total,   1 running,  13 sleeping,   0 stopped,   9 zombie
%Cpu(s):  1.8 us,  0.9 sy,  0.0 ni, 97.0 id,  0.3 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 13305368 total,  9552540 free,   772080 used,  2980748 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 12270004 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND   
      1 root      20   0  346468  57948  32632 S   6.7  0.4   0:06.47 node      
     16 root      20   0   35888   4856   3756 S   0.0  0.0   0:00.64 tail      
     51 root      20   0  195184  61700  13656 S   0.0  0.5   0:07.20 jupyter-+ 
     52 root      20   0  709868  10936   4932 S   0.0  0.1   0:05.09 dap_mult+ 
     82 root      20   0       0      0      0 Z   0.0  0.0   0:00.13 python3   
    132 root      20   0 2083536 451944 196764 S   0.0  3.4   0:23.41 python3   
    150 root      20   0  128668  16716   5872 S   0.0  0.1   

In [ ]:
!kill 1296